In [80]:
from langchain_huggingface import HuggingFaceEmbeddings

print("Ok")

Ok


In [81]:
%pwd

'F:\\End-to-End-Medical-Chat-Bot\\research'

In [82]:
import os
os.chdir("../")

In [83]:
%pwd

'F:\\End-to-End-Medical-Chat-Bot'

In [84]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [4]:
extracted_data = load_pdf_file(data=r"F:\End-to-End-Medical-Chat-Bot\Data")


In [6]:
#extracted_data

In [7]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text__chunks = text_splitter.split_documents(extracted_data)
    return text__chunks

In [8]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks" , len(text_chunks))

Length of Text Chunks 6973


In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

In [14]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [85]:
embeddings = download_hugging_face_embeddings()

In [17]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [37]:
from dotenv import load_dotenv
load_dotenv()

True

In [38]:
import os
PINECONE_API_KEY=os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY=os.environ.get("OPENAI_API_KEY")

In [48]:
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index_name = "medibot"

# check if index already exists
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=384,  # must match your embedding model output size
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# connect to the existing index
index = pc.Index(index_name)


In [49]:
import os
os.environ["PINECONE_API_KEY"]= PINECONE_API_KEY
os.environ["OPENAI_API_KEY"]= OPENAI_API_KEY

In [50]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [51]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [52]:
docsearch

In [53]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [54]:
retrieved_docs = retriever.invoke("What is Acne?")

In [55]:
retrieved_docs


[Document(metadata={'page': 425.0, 'source': 'F:\\End-to-End-Medical-Chat-Bot\\Data\\Medical_Book.pdf'}, page_content='Corticosteriod —A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-'),
 Document(metadata={'page': 298.0, 'source': 'F:\\End-to-End-Medical-Chat-Bot\\Data\\Medical_Book.pdf'}, page_content='Corticosteroids —A group of anti-inflammatory\nsubstances often used to treat skin conditions.\nImmune response—The protective reaction by the\nimmune system against foreign antigens (sub-\nstances that the body perceives as potentially dan-\ngerous). The immune system combats disease 

In [56]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [68]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you don't know. "
    "Use three sentences maximum and keep the answer concise.\n\n{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)




In [59]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)


In [79]:
response = rag_chain.invoke({"input" : "What are the medications for heart disease?"})
print(response["answer"])



ACE inhibitors, anticoagulant medications, medicines to prevent abnormal heart rhythms, amphetamines, medicine to relieve asthma or other breathing problems, methylphenidate (Ritalin), appetite suppressants, other medicine for colds, sinus problems, hay fever or other allergies, beta-blockers such as atenolol (Tenormin) and propranolol (Inderal), digitalis glycosides, antihistamines, antipsychotic drugs, aspirin, belladonna (Atropa belladonna), beta blockers, blood pressure medications, and calcium channel blockers.
